# FTGP Week 2: Blockchain fundamentals

This week, we will use the cryptographic primitives we saw last week to build our own blockchain!

Understanding how a typical blockchain works is a key part of understanding the benefits (and drawbacks) this technology brings. Knowing how the underling systems works, will also improve your ability to build financial applications on top of it.

We will be using Python and the packages we used last week, plus you might the following packages: `json`, `cbor` (https://docs.python.org/3/library/json.html, https://cbor2.readthedocs.io/en/latest/)

***Note:*** The following worksheet requires good programming skills. However, the aim of the worksheet is not to teach you how to code, but to allow you to build your own blockchain system. If you are not feeling confident in your programming abilities, you might wish to use a non-programming alternative, ETH.build (https://sandbox.eth.build/). A series of helpful videos (https://www.youtube.com/playlist?list=PLJz1HruEnenCXH7KW7wBCEBnBLOVkiqIi) is also available to guide you throughout the process.


### Question 1: Transactions

Design and implement a `Transaction` class that includes the following fields

- sender's public key
- receiver's public key
- transaction amount (integer >0)
- comment (arbitrary text, can be empty)
- sender's signature (signing the transaction data)

Your class should also implement the following methods:

- new: to create a new transaction
- sign: to sign the transaction data and produce a valid signature
- validate: to validate a transaction
- serialize: to turn a transaction object into a shareable format (using CBOR or JSON)
- deserialize: to turn a serialized transaction into an object

The following skeleton code might be helpful.

What checks are you going to include within `validate()` ?

In [ ]:
import time
import math
import hashlib
import ecdsa
import cbor2


class Transaction():

    def __init__(self):
        self.data = {
            "sender" : None,
            "receiver" : None,
            "amount" : None,
            "comment" : None,
            "signature" : None,
        }
    
    def __eq__(self, check):
        if ((self.data["sender"] == check.data["sender"] ) and (self.data["receiver"] == check.data["receiver"] ) and (self.data["amount"] == check.data["amount"]) and (self.data["comment"] == check.data["comment"]) and (self.data["signature"] == check.data["signature"])):
            return True
        else:
            return False

    def new(self, sender, receiver, amount, comment = ""):
        # Create transaction from passed values
        # Hint: store the keys as byte strings


    def sign(self, sk):
        # Sign transaction data with private key
        # Hint: you fist need to serialize the transaction data


    def validate(self, public_key, signature):
        # Validate transaction correctness (i.e. valid signature)


    def serialize(self):
        # Serializes object to CBOR or JSON string


    def deserialize(self, obj):
        # Deserializes object from CBOR or JSON string
        # Make sure that the transaction is valid
        # Hint: what should the signature be when validating the transaction?
        

Test your implementation.

Do you think it makes sense to add any other fields/functions?

### Question 2: 

For this question we will need the Merkle tree we implemented last week. An implementation is provided below, but feel free to use your own implementation from last week.

In [ ]:
class MerkleTree():

    def __init__(self):
        self.leaves = [] # entries are assumed to be bytes
        self.hashes = []
        self.root = None

    def add(self, entry):
        # Add entries to tree 
        
        self.leaves += [entry]
        self.hashes += [hashlib.sha256(entry).digest()]

    def build(self):
        # Build tree computing new root

        level = math.ceil(math.log(len(self.hashes), 2)) # next power of two
        if 2**level != len(self.hashes): # if not complete tree
            for i in range(2**level - len(self.hashes)): # add empty leaves so that the tree is complete
                self.add(b"")

        for i in range((2**level)-1): # calculate the rest of the nodes
            self.hashes += [hashlib.sha256(self.hashes[2*i] + self.hashes[2*i+1]).digest()]
        
        self.root = self.hashes[-1] # set root

    def get_proof(self, entry):
        # Get membership proof for a given entry

        if entry not in self.leaves:
            return None
        else:
            degree = math.log(len(self.leaves), 2)
            index = self.leaves.index(entry)
            proof = []
            while self.hashes[index] != self.root:
                if index % 2 == 0:
                    proof += [(self.hashes[index+1], 'r')] # add right sibling
                else:
                    proof += [(self.hashes[index-1], 'l')] # add left sibling
                index = int(2**degree + math.floor(index/2)) # go to parent
            return proof

    def get_root(self):
        # Return the current root

        if self.root == None:
            return b"0"
        else:
            return self.root

Design and implement a `Block` class. Each `Block` object should have:

- set of serialized transactions that form a Merkle tree
- header that includes:
    - block number
    - hash of the previous block's header
    - root of the hash tree
    - timestamp (Unix timestamp expressed as an integer)
    - nonce (a random number needed to generate PoW)
- methods:
    - new: Create a new block from passed values
    - get_hash: Return the hash of the provided values
    - addNonce: Add nonce to block header
    - serialize: Serializes object to CBOR or JSON string

To make this a bit easier, at this point you don't have to implement any checks for the validity of the included transactions.

In [ ]:
class Block():

    def __init__(self):
        self.txs = None
        self.tree = None
        self.header = {
            "number"    : None,
            "prev_hash" : None,
            "root_hash" : None,
            "timestamp" : None,
            "nonce"     : None
        }

    def new(self, txs, number, prev_hash, timestamp):
        # Create a new block from passed values
        # Hint: you can assume that the transactions are already serialized
        

    def get_hash(self, number, prev_hash, root_hash, timestamp, nonce):
        # Return the hash of the provided values
        # The format should be something like: hash = H(number|prev_hash|root|time|nonce)

        
    def block_hash(self):
        # Return the hash of the block header


    def addNonce(self, nonce):
        # Add nonce to block header
        

    def serialize(self):
        # Serializes object to CBOR or JSON string


Test your implementation.

### Question 3: 

Design and implement the `Blockchain` class. A `Blockchain` object contains `Block` object(s). Each `Blockchain` object should:

- store the genesis block
- keep track of all blocks
- methods:
    - add: Add a new block to the blockchain
    - validate: Validate a block before adding it (for now only perform checks on the block header)
    - last_hash: Returns the hash of the latest block
    - last_number: Returns the number of the latest block

In [ ]:
class Blockchain():

    def __init__(self):
        self.head = None
        self.blocks = []
    
    def add(self, node):
        # Add a new block to the chain


    def validate(self, node):
        # Validate a block


    def last_hash(self):
        # Return the hash of the last block


    def last_number(self):
        # Return the number of the last block
        

Test your implementation.

What checks have you implemented in `Blockchain`'s `validate()`?

### Bonus Question 4:

Congratulations! If you have reached this point, you have implemented your own blockchain system!

However, at the current state, our blockchain is missing a few features. Introduce:
- The Coinbase transaction. Introduce a new special transaction, where the miner who found a new block is rewarded with 100 coins. 
- Make sure that only one such transaction can be included per block.
- PoW. The hash of every new block's header should be less than `TARGET` (a global parameter, set now to `\x00\x0f\xff\xff\xff\xff...\xff`). 
- Write a function that finds the appropriate block nonce for PoW.
- Forks and their handling. Modify your implementation, such that `add()` allows to anchor a new block to any given existing block. 
- Implement the `resolve()` method, that returns the longest chain (e.g., it can return the latest block of the longest chain). 
- At the moment, our blockchain system doesn't perform any checks on the transactions (apart from verifying the signature). Modify your block implementation so that each block contains a record of user accounts (i.e. mapping from public keys to balances). 
- Then create a method that makes sure that a transaction is valid (given the block's user record) before adding it to the block. Otherwise the transaction should be discarded without any state changes.
- At the moment, transactions can be repeated. Add a counter associated with each account that makes each transaction unique.
- Finally, add a method to your blockchain class to validate that the accounts (balances and counters) of each block. Make sure that the accounts are matching the ones in the previous block after the transactions of the block.